In [14]:
from selenium import webdriver
import time, re, imaplib, email
import base64
from bs4 import BeautifulSoup
import random

import psycopg2
conn = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="5432", database="postgres")
cur = conn.cursor()

import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('gmail', 'v1', credentials=creds)

def get_code_from_gmail():
    results = service.users().messages().list(userId='me',labelIds = ['INBOX']).execute()
    messages = results.get('messages', [])
    if not messages:
        print("No messages found.")
    else:
        # https://developers.google.com/gmail/api/v1/reference/users/messages?refresh=1
        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            body = msg['payload']['parts'][1]['body']['data']
            contents = base64.urlsafe_b64decode(body)
            soup = BeautifulSoup(contents, 'html.parser')
            return(int(soup.find_all('h2')[1].text.strip()))
        
def get_code_from_hotmail():
    mail = imaplib.IMAP4_SSL('smtp-mail.outlook.com')
    mail.login('rajatpundir13@hotmail.com', '1nst1nct')
    mail.list()
    mail.select("inbox")
    result, data = mail.search(None, "ALL")
    ids = data[0]
    id_list = ids.split()
    latest_email_id = id_list[-1]
    result, data = mail.fetch(latest_email_id, "(RFC822)") 
    raw_email = data[0][1]
    msg = email.message_from_bytes(raw_email)
    body = str(msg.get_payload()[0].get_payload(decode=True))
#     print(msg['from'], msg['subject'])
    return(int(re.findall('(\d{6})', body)[0]))

driver = webdriver.Chrome()
# driver = webdriver.Firefox()

driver.get('https://www.freelancer.com/jobs/')
time.sleep(1)
login_button_1 = driver.find_element_by_xpath('/html/body/div[1]/header/div/div/div[2]/fl-login-signup-angular/a[1]')
login_button_1.click()

time.sleep(1)

In [28]:
cur.execute("SELECT id, password, username from users;")
r = random.choice(cur.fetchall())
print(r)

('akshay.kulkarni@pundir.solutions', 'fp4dTf', 'akulkarni227')


In [15]:
var_username = r[0]
var_pass = r[1]
username = driver.find_element_by_xpath('//*[@id="username"]')
password = driver.find_element_by_xpath('//*[@id="password"]')
username.clear()
username.send_keys(var_username)
password.clear()
password.send_keys(var_pass)
remember_me = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-login-form/div[2]/form[2]/fieldset/ol/li[3]/div/label')
# remember_me.clear()
remember_me.click()
login_button_2 = driver.find_element_by_xpath('//*[@id="login_btn"]')
login_button_2.click()
time.sleep(8)
give_me_code = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-totp-form/div[2]/div/form/fieldset/ol/li[2]/input')
give_me_code.send_keys(get_code_from_hotmail())

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-totp-form/div[2]/div/form/fieldset/ol/li[2]/input"}
  (Session info: chrome=74.0.3729.108)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17763 x86_64)


In [17]:
# try:
prev_count = 0
while True:
    driver.find_element_by_xpath('//*[@id="clear-btn"]').click()
    time.sleep(4)
    cur.execute("SELECT count(*) from projects;")
    count = cur.fetchone()[0]
    if count != prev_count:
        print('total : ', count)
        prev_count = count
    rows = driver.find_element_by_xpath('//*[@id="project_table"]/tbody')
    soup = BeautifulSoup(rows.get_attribute('innerHTML'), 'html.parser')
    for i in range(len(soup.find_all('tr'))):
        row = soup.find_all('tr')[i]
        if row.text == 'No projects or contests found.':
            print('break')
            break
        tags = []
        for j in range(len(row.find_all('span')[1].find_all('a'))):
            tags.append('\'' + row.find_all('span')[1].find_all('a')[j].text.strip() + '\'')
        url = row.find_all('a')[0]['href']
        project_id = int(row['id'])
        body = row.find_all('td')[0].find_all('p').pop().text.strip()
        bid_count = re.findall(r'\d+', row.find_all('td')[1].text.strip())[0]
        time_left = row.find_all('td')[2].find_all('small').pop().text
        money = row.find_all('td')[4].find_all('span')[0].text.strip()
        cur.execute("SELECT id FROM projects where id = %s;", (project_id,))
        if cur.fetchone() is None:
            cur.execute("INSERT INTO projects (id, bid_count, url, body, time_left, money, time) VALUES (%s, %s, %s, %s, %s, %s, %s)", (project_id, bid_count, url, body, time_left, money, time.time()))
            conn.commit()
            cur.execute("INSERT INTO tags (id, tag) VALUES (%s, ARRAY[%s])", (project_id, ', '.join(tags)))
            header = 'new'
            if('\'Excel\'' in tags):
                header = 'Excel'
            if('\'Python\'' in tags):
                header = 'Python'
            print(header, bid_count, url, money)
        else:
            cur.execute("SELECT bid_count from projects where id=%s", (project_id,))
            r = cur.fetchone()
            if r[0] != int(bid_count):
                cur.execute("UPDATE projects SET bid_count=%s, url=%s, body=%s, time_left=%s, money=%s WHERE id=%s", (bid_count, url, body, time_left, money, project_id))
                cur.execute("SELECT bid_count, time, url from projects where id=%s", (project_id,))
                r = cur.fetchone()
                print(r[0], time.time() - r[1], r[2])
    conn.commit()
    driver.refresh()
    time.sleep(4)
# except:
#     conn.commit()
#     print('success')

total :  342
new 0 https://www.freelancer.com/projects/research-writing/economics-based-project/ ₹600 - ₹1503
Excel 0 https://www.freelancer.com/projects/data-entry/looking-for-freelancers-who-can-19427839/ ₹554 - ₹1041/hr Unspecified
new 0 https://www.freelancer.com/projects/website-design/build-really-attractive-website-based/ ₹600 - ₹1503
Python 9 https://www.freelancer.com/projects/php/php-python-code-migration/ ₹9156
new 1 https://www.freelancer.com/projects/html/react-app-simple-refactoring-amend/ ₹1387
new 4 https://www.freelancer.com/projects/internet-marketing/lead-generator-expert-linkedin/ ₹1249 / hrUnspecified
new 7 https://www.freelancer.com/projects/php/adapt-existing-site-new-graphical/ ₹36014
new 0 https://www.freelancer.com/projects/mobile-phone/want-use-webcam-nox-player/ ₹693 - ₹2082
new 3 https://www.freelancer.com/projects/html/react-experts-for-web/ ₹1288
new 1 https://www.freelancer.com/projects/android/france-local-testing-mobile-campaign/ ₹2289
new 4 https://ww

34 125.68637752532959 https://www.freelancer.com/projects/graphic-design/graphic-design-adverts-19427689/
total :  402
Excel 0 https://www.freelancer.com/projects/data-entry/need-specific-data-from-website/ ₹2081 - ₹17342
new 1 https://www.freelancer.com/projects/logo-design/hip-hop-banger/ ₹193622
10 65.9377920627594 https://www.freelancer.com/projects/internet-marketing/press-release-distribution-19427860/
8 130.94131469726562 https://www.freelancer.com/projects/website-design/build-website-19427856/
144 130.94131469726562 https://www.freelancer.com/projects/php/redesign-responsive-website/
20 130.95151257514954 https://www.freelancer.com/projects/graphic-design/index-website-with-google/
2 190.96147680282593 https://www.freelancer.com/projects/research-writing/economics-based-project/
3 190.9714171886444 https://www.freelancer.com/projects/website-design/build-really-attractive-website-based/
8 190.9915759563446 https://www.freelancer.com/projects/internet-marketing/lead-generator-e

total :  414
8 103.69909453392029 https://www.freelancer.com/projects/php/onlineshop/
11 172.69911766052246 https://www.freelancer.com/projects/articles/new-version-press-release/
15 172.7090768814087 https://www.freelancer.com/projects/php/redirect-part-the-website/
6 172.71912360191345 https://www.freelancer.com/projects/javascript/mysql-specialist/
0 172.72944498062134 https://www.freelancer.com/projects/engineering/agricultural-machinery-project-19427875/
10 248.72944498062134 https://www.freelancer.com/projects/data-entry/need-specific-data-from-website/
6 314.73938059806824 https://www.freelancer.com/projects/excel/confirmation/
11 314.74914050102234 https://www.freelancer.com/projects/internet-marketing/press-release-distribution-19427860/
146 379.7595250606537 https://www.freelancer.com/projects/php/redesign-responsive-website/
25 379.769419670105 https://www.freelancer.com/projects/graphic-design/index-website-with-google/
2 439.77911281585693 https://www.freelancer.com/projec

total :  426
Excel 0 https://www.freelancer.com/projects/data-entry/part-time-admin-assistant-with-19427947/ ₹1039 - ₹1733/hr Unspecified
new 0 https://www.freelancer.com/projects/mobile-phone/material-sole-material-pvc-eva/ ₹1501 - ₹12509
new 0 https://www.freelancer.com/projects/content-writing/ghost-stories/ ₹599 - ₹1500
Python 0 https://www.freelancer.com/projects/php/need-tron-roi-smart-contract/ ₹100 - ₹401/hr Unspecified
9 68.67260146141052 https://www.freelancer.com/projects/graphic-design/instagram-content-creation-19427943/
1 68.67260146141052 https://www.freelancer.com/projects/video-services/video-spot-19427942/
4 68.68272042274475 https://www.freelancer.com/projects/website-design/website-messaging-revamp/
2 139.68272042274475 https://www.freelancer.com/projects/data-entry/fluent-english-speech/
8 139.69261193275452 https://www.freelancer.com/projects/data-entry/data-entry-work-19427932/
11 139.69261193275452 https://www.freelancer.com/projects/data-entry/need-excel-expert

IndexError: pop from empty list

In [31]:
row.text == 'No projects or contests found.'

True